In [2]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import torch

import pandas as pd

In [3]:
from tqdm.notebook import tqdm
import sys
import os
import re
from glob import glob
from time import sleep

prefix = os.getcwd().split("jdi-qasp-ml")[0]
sys.path.append(os.path.join(prefix, "jdi-qasp-ml"))

In [4]:
dataset_path = os.path.join(prefix, "jdi-qasp-ml", "data/mui_dataset")

SITE_DFS = [
    p
    for p in glob(f'{os.path.join(prefix, "jdi-qasp-ml")}/data/mui_dataset/df/site*')
]
DF_NAMES = [re.search("site-[0-9]+", nm)[0] for nm in SITE_DFS]

In [5]:
# total_df = pd.DataFrame()
# for i, df_path in tqdm(enumerate(SITE_DFS)):
#     df = pd.read_pickle(df_path)
#     df['site'] = DF_NAMES[i]

#     par_childs = {par:list(df[df.parent_id==par].element_id.unique()) for par in df.parent_id.unique()}

#     def find_all_childs(elem_id):
#         if par_childs.get(elem_id) is None:
#             return []
#         else:
#             res = []
#             for i in par_childs.get(elem_id):
#                 res.append(i)
#                 res.extend(find_all_childs(i))
#             return res
    
#     df['all_followers'] = df['element_id'].apply(find_all_childs)

#     df['STRING'] = df.attributes.apply((lambda x: '' if x is None else x.get('class', "") + " " + x.get('type', "") + " " + x.get('role', "")))
#     df['STRING'] = df.tag_name.str.lower().str.cat(df['STRING'], sep=" ")
#     df['LABEL'] = df.attributes.apply(lambda x: None if x is None else x.get('data-label'))
#     df = df[["site", "element_id", "parent_id", "all_followers", "STRING", "LABEL"]]
#     df.reset_index(drop=True, inplace=True)

#     elem_string_dict = {k:v.get('STRING') for k,v in df[['STRING']].set_index(df.element_id).fillna('').T.to_dict().items()}

#     def concat_all_strings(list):
#         res = ''
#         for l in list:
#             res = res + elem_string_dict.get(l) + ' '
#         return res
    
#     df['all_followers_string'] = df['all_followers'].apply(concat_all_strings)
#     df['final_string'] = (df.STRING + ' ' + df.all_followers_string).fillna('')

#     df = df[["site", "element_id", "parent_id", "final_string", "LABEL"]]
#     total_df = pd.concat([total_df, df])
# total_df.reset_index(inplace=True, drop=True)

In [6]:
# total_df.tail(3)

In [7]:
# total_df.to_csv(f'{os.path.join(prefix, "jdi-qasp-ml")}/data/other/total_df_for_nlp_model.csv', index=False)

In [8]:
# total_df = pd.read_csv(f'{os.path.join(prefix, "jdi-qasp-ml")}/data/other/total_df_for_nlp_model.csv')
# total_df.LABEL = total_df.LABEL.fillna("n/a")


In [9]:
# total_df.tail()

In [10]:
def prepare_string(string):
    string = re.sub(r"(?<=\w)([A-Z])", r" \1", string)
    string = str(string).lower().strip()
    return(re.sub(' +', ' ', re.sub('(-)|([0-9]+)', ' ', string).strip()))


# total_df['final_string'] = total_df['final_string'].apply(prepare_string)

In [11]:
# total_df.tail()

In [12]:
# total_df.to_csv(f'{os.path.join(prefix, "jdi-qasp-ml")}/data/other/total_df_for_nlp_model_prepared.csv', index=False)

In [13]:
total_df = pd.read_csv(f'{os.path.join(prefix, "jdi-qasp-ml")}/data/other/total_df_for_nlp_model_prepared.csv')
total_df.LABEL = total_df.LABEL.fillna("n/a")
total_df.LABEL.replace("n/a", "unkn", inplace=True)

In [14]:
total_df.tail()

,site,element_id,parent_id,final_string,LABEL
739896,site-13,9627524316559287764950874349,8293726446559287768833831671,div,unkn
739897,site-13,9638773789559287760038756767,8293726446559287768833831671,div,unkn
739898,site-13,4889127663559287761125737880,8293726446559287768833831671,div mui paper root mui popover paper mui paper...,unkn
739899,site-13,8651010676559287765452773916,4889127663559287761125737880,p mui typography root jss mui typography body,typography
739900,site-13,9980562521559287761155767932,8293726446559287768833831671,div,unkn


In [15]:
train_dfs = DF_NAMES[:50]
val_dfs = DF_NAMES[400:450]
test_dfs = DF_NAMES[450:]

train_df = total_df[total_df.site.isin(train_dfs)].copy()[['element_id', 'final_string', 'LABEL']].copy().reset_index(drop=True)
val_df = total_df[total_df.site.isin(val_dfs)].copy()[['element_id', 'final_string', 'LABEL']].copy().reset_index(drop=True)
test_df = total_df[total_df.site.isin(test_dfs)].copy()[['element_id', 'final_string', 'LABEL']].copy().reset_index(drop=True)

# train.to_csv(f'{os.path.join(prefix, "jdi-qasp-ml")}/data/other/'+ 'nlp_train.csv', index=False)
# val_df.to_csv(f'{os.path.join(prefix, "jdi-qasp-ml")}/data/other/'+ 'nlp_val.csv', index=False)
# test_df.to_csv(f'{os.path.join(prefix, "jdi-qasp-ml")}/data/other/'+ 'nlp_test.csv', index=False)

In [16]:
train_df[~(train_df.LABEL == "unkn")]

,element_id,final_string,LABEL
112,6533486216559833460616205753,span mui slider root mui slider color primary ...,slider
126,7159805062559833463754904441,div mui avatar root mui avatar square mui avat...,avatar
127,9909531984559833468390915811,div mui avatar root mui avatar square mui avat...,avatar
128,8242725098559833466945321206,svg mui svg icon root path,icon
130,3265653768559833466368332963,div mui avatar root mui avatar square mui avat...,avatar
...,...,...,...
73190,7140654154569241153037853244,ul mui list root mui menu list mui list paddin...,menu
73198,9907290597569241159943579524,div tooltip div mui paper root mui paper eleva...,popper
73199,6530051995569241159732622655,div mui paper root mui paper elevation mui pap...,paper
73200,1575209453569241157856032962,p mui typography root jss mui typography body,typography


In [17]:
train_df[train_df.element_id=="6533486216559833460616205753"]

,element_id,final_string,LABEL
112,6533486216559833460616205753,span mui slider root mui slider color primary ...,slider


Create text pipeline

In [18]:
import itertools
tokenizer = get_tokenizer('basic_english')
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(prepare_string(text))

vocab = build_vocab_from_iterator(yield_tokens(train_df.final_string))

In [19]:
print(len(vocab))
print(vocab['accordion'])

273
59


In [20]:
classes_path = os.path.join(prefix, "jdi-qasp-ml", "data/mui_dataset/classes.txt")
with open(classes_path, "r") as f:
    lines = f.readlines()
    classes_dict = {v.strip(): i for i, v in enumerate(lines)}
    classes_reverse_dict = {i: v.strip() for i, v in enumerate(lines)}

# train_df.LABEL = train_df.LABEL.apply(lambda x: classes_dict.get(x, 0))

In [21]:
classes_dict["slider"]

23

In [22]:
text_pipeline = lambda x: vocab(tokenizer(prepare_string(x)))
label_pipeline = lambda x: classes_dict.get(x, 0)

In [23]:
text_pipeline("div MuiPaper-root MuiPopover-paper MuiPaper")

[7, 0, 38, 1, 0, 163, 38, 0, 38]

Generate data batch and iterator

In [24]:
from torchtext.legacy import data

In [25]:
from torch.utils.data import DataLoader
device = torch.device("cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [26]:
TEXT = data.Field(include_lengths=True, sequential=True, tokenize=tokenizer, lower=True, use_vocab=True)
LABEL = data.Field(sequential=False, use_vocab=True)
ID = data.RawField()

In [27]:
from torchtext.legacy import data

class DataFrameDataset(data.Dataset):

    def __init__(self, df, id_field, text_field, label_field, is_test=False, **kwargs):
        fields = [('id', id_field), ('text', text_field), ('label', label_field)]
        examples = []
        for i, row in df.iterrows():
            label = row.LABEL if not is_test else None
            text = row.final_string
            id = row.element_id
            examples.append(data.Example.fromlist([id, text, label], fields))

        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex):
        return len(ex.text)

    @classmethod
    def splits(cls, id_field, text_field, label_field, train_df, val_df=None, test_df=None, **kwargs):
        train_data, val_data, test_data = (None, None, None)

        if train_df is not None:
            train_data = cls(train_df.copy(), id_field, text_field, label_field, **kwargs)
        if val_df is not None:
            val_data = cls(val_df.copy(), id_field, text_field, label_field, **kwargs)
        if test_df is not None:
            test_data = cls(test_df.copy(), id_field, text_field, label_field, True, **kwargs)

        return tuple(d for d in (train_data, val_data, test_data) if d is not None)
        
train_ds, val_ds, test_ds = DataFrameDataset.splits(id_field=ID, text_field=TEXT, label_field=LABEL, train_df=train_df, val_df=val_df, test_df=test_df)

In [28]:
print(train_ds[112].id)
print(train_ds[112].label)
print(train_ds[112].text)

6533486216559833460616205753
slider
['span', 'mui', 'slider', 'root', 'mui', 'slider', 'color', 'primary', 'span', 'mui', 'slider', 'rail', 'span', 'mui', 'slider', 'track', 'input', 'hidden', 'span', 'mui', 'slider', 'thumb', 'mui', 'slider', 'thumb', 'color', 'primary', 'jss', 'jss', 'slider', 'span', 'jss', 'mui', 'slider', 'value', 'label', 'span', 'jss', 'span', 'jss', 'span', 'mui', 'slider', 'thumb', 'mui', 'slider', 'thumb', 'color', 'primary', 'jss', 'jss', 'slider', 'span', 'jss', 'mui', 'slider', 'value', 'label', 'span', 'jss', 'span', 'jss']


In [29]:
print("\nCreating vocabulary dict from train data object ")
TEXT.build_vocab(train_ds.text)
print("The idx of \'mui\' is ", TEXT.vocab.stoi['paper'])
print("The string value of 4 is ", TEXT.vocab.itos[4])


Creating vocabulary dict from train data object 
The idx of 'mui' is  40
The string value of 4 is  button


In [30]:
print("\nCreating vocabulary dict from train data object ")
LABEL.build_vocab(train_ds.label)
print("The idx of \'slider\' is ", LABEL.vocab.stoi['slider'])
print("The string value of 4 is ", LABEL.vocab.itos[4])


Creating vocabulary dict from train data object 
The idx of 'slider' is  34
The string value of 4 is  typography


In [31]:
print("\nCreating a BucketIterator on the train_object ")

train_iter  = data.BucketIterator(
  dataset=train_ds,
  batch_size = 1024,
  sort_key=lambda x: len(x.text),
  shuffle=False,
  device=device)

val_iter  = data.BucketIterator(
  dataset=val_ds,
  batch_size = 1024,
  sort_key=lambda x: len(x.text),
  shuffle=False,
  device=device)

print("\nIterating train data (batch_size=32) ")
for item in train_iter:
  # print("\n=====\n")
  # print(item.id)
  # print(item.text)
  # print(item.label)
  if '6533486216559833460616205753' in item.id:
    break


Creating a BucketIterator on the train_object 

Iterating train data (batch_size=32) 


In [32]:
item.text[0].T

tensor([[273,  96, 196,  ..., 245,   9,  11],
        [ 96, 196,  84,  ...,   1,   1,   1],
        [196,   1,   1,  ...,   1,   1,   1],
        ...,
        [  9,  11,   9,  ...,   1,   1,   1],
        [  9,   2,  40,  ...,   1,   1,   1],
        [  9,   2, 118,  ...,   1,   1,   1]])

In [33]:
[TEXT.vocab.itos[i] for i in item.text[0][:,0]]

['html',
 'head',
 'meta',
 'link',
 'meta',
 'meta',
 'meta',
 'link',
 'link',
 'title',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',
 'style',

In [34]:
item.id[0]

'5377782770559833469033671578'

In [56]:
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import torch.optim as optim

# Evaluation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

class LSTM(nn.Module):

    def __init__(self, dimension=32):
        super(LSTM, self).__init__()

        self.embedding = nn.Embedding(len(TEXT.vocab), 64)
        self.dimension = dimension
        self.lstm = nn.LSTM(input_size=64,
                            hidden_size=dimension,
                            num_layers=1,
                            batch_first=True,
                            bidirectional=True)
        self.drop = nn.Dropout(p=0.5)

        self.fc = nn.Linear(2*dimension, len(LABEL.vocab), bias=False)

    def forward(self, text, text_len):

        text_emb = self.embedding(text)

        packed_input = pack_padded_sequence(text_emb, text_len, batch_first=True, enforce_sorted=False)
        packed_output, _ = self.lstm(packed_input)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)

        out_forward = output[range(len(output)), text_len - 1, :self.dimension]
        out_reverse = output[:, 0, self.dimension:]
        out_reduced = torch.cat((out_forward, out_reverse), 1)
        text_fea = self.drop(out_reduced)

        text_out = self.fc(text_fea)

        return text_out

In [57]:
def save_checkpoint(save_path, model, optimizer, valid_loss):

    if save_path == None:
        return
    
    state_dict = {'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'valid_loss': valid_loss}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


def load_checkpoint(load_path, model, optimizer):

    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    model.load_state_dict(state_dict['model_state_dict'])
    optimizer.load_state_dict(state_dict['optimizer_state_dict'])
    
    return state_dict['valid_loss']


def save_metrics(save_path, train_loss_list, valid_loss_list, global_steps_list):

    if save_path == None:
        return
    
    state_dict = {'train_loss_list': train_loss_list,
                  'valid_loss_list': valid_loss_list,
                  'global_steps_list': global_steps_list}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


def load_metrics(load_path):

    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    return state_dict['train_loss_list'], state_dict['valid_loss_list'], state_dict['global_steps_list']

In [58]:
dest_folder = f'{os.path.join(prefix, "jdi-qasp-ml")}/data/other/'

In [59]:
from tqdm.notebook import tqdm

In [60]:
def train(model,
          optimizer,
          criterion = nn.CrossEntropyLoss(),
          train_loader = train_iter,
          valid_loader = val_iter,
          num_epochs = 5,
          eval_every = 1000,
          file_path = dest_folder,
          best_valid_loss = float("Inf")):
    
    # initialize running values
    running_loss = 0.0
    valid_running_loss = 0.0
    global_step = 0
    train_loss_list = []
    valid_loss_list = []
    global_steps_list = []

    # training loop
    model.train()
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}!')
        for batch in tqdm(train_loader):           
            labels = batch.label.T.long().to(device)
            print(labels)
            text = batch.text[0].T.to(device)
            # print(text.shape)
            titletext_len = batch.text[1].T.to(device)
            # print(titletext_len.shape)
            output = model(text, titletext_len)
            print(output)
            optimizer.zero_grad()
            print("zerograd")
            loss = criterion(output, labels)
            print("loss") 
            return labels, text, output           
            loss.backward()
            print("backward")
            optimizer.step()
            print("optimizer step")
            # update running values
            running_loss += loss.item()
            global_step += 1

            # evaluation step
            if global_step % eval_every == 0:
                model.eval()
                with torch.no_grad():                    
                  # validation loop
                  print("Validation loop")
                  for batch in valid_loader:
                      labels = batch.label.T.long().to(device)
                      text = batch.text[0].T.to(device)
                      titletext_len = batch.text[1].T.to(device)
                      output = model(text, titletext_len)

                      loss = criterion(output, labels)
                      valid_running_loss += loss.item()

                # evaluation
                average_train_loss = running_loss / eval_every
                average_valid_loss = valid_running_loss / len(valid_loader)
                train_loss_list.append(average_train_loss)
                valid_loss_list.append(average_valid_loss)
                global_steps_list.append(global_step)

                # resetting running values
                running_loss = 0.0                
                valid_running_loss = 0.0
                model.train()

                # print progress
                print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}'
                      .format(epoch+1, num_epochs, global_step, num_epochs*len(train_loader),
                              average_train_loss, average_valid_loss))
                
                # checkpoint
                if best_valid_loss > average_valid_loss:
                    best_valid_loss = average_valid_loss
                    save_checkpoint(file_path + '/model.pt', model, optimizer, best_valid_loss)
                    save_metrics(file_path + '/metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    
    save_metrics(file_path + '/metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    print('Finished Training!')

In [61]:
model = TextClassificationModel(len(TEXT.vocab), 128, len(LABEL.vocab)).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [62]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(model)

40489

In [63]:
labels, text, output = train(model=model, optimizer=optimizer, num_epochs=10)

Epoch 0!


  0%|          | 0/72 [00:00<?, ?it/s]

tensor([ 1,  1,  1,  ...,  1, 29,  1])


ValueError: if input is 2D, then offsets has to be None, as input is treated is a mini-batch of fixed length sequences. However, found offsets of type <class 'torch.Tensor'>

In [53]:
print(list(labels))

[tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1)

In [67]:
text[-2].shape

torch.Size([7588])

In [ ]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)

In [ ]:
input.shape

torch.Size([3, 5])

In [ ]:
target.shape

torch.Size([3])

In [ ]:
loss(input, target)

tensor(2.3169, grad_fn=<NllLossBackward0>)